In [ ]:
#作用：将所有生成的特征进行切割，按顺序划分五折，保存五份训练集和验证集，均以h5py格式进行保存，加快模型训练读写速度
#注：代码在多个服务器上均有运行，部分路径可能有修改，如有问题，及时联系

In [ ]:
!pip install gensim
!pip install ipywidgets==5.0.0
!pip uninstall numpy -y
!pip install numpy==1.16.2
!pip uninstall keras -y
!pip install keras==2.2.4

In [10]:
# 导入相关库
import os
import pandas as pd
from tqdm.autonotebook import *
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.metrics import accuracy_score
import time
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
from sklearn.model_selection import StratifiedKFold
from gensim.models import FastText, Word2Vec
import re
from keras.layers import *
from keras.models import *
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import text, sequence
from keras.callbacks import *
from keras.layers.advanced_activations import LeakyReLU, PReLU
import keras.backend as K
from keras.optimizers import *
from keras.utils import to_categorical
import tensorflow as tf
import random as rn
import gc
import logging
import gensim
from sklearn.preprocessing import MinMaxScaler
os.environ['PYTHONHASHSEED'] = '0'
np.random.seed(1024)
rn.seed(1024)
tf.set_random_seed(1024)

In [ ]:
x1 = np.load('./w2v_model1_100_256/x1.npy')
x2 = np.load('./w2v_model2_100_256/x1.npy')
x3 = np.load('./w2v_model3_100_128/x1.npy')
x4 = np.load('./w2v_model4_100_128/x1.npy')
x5 = np.load('./w2v_model5_100_64/x1.npy')

In [3]:
labels = pd.read_pickle('label.pkl')

In [16]:
labels.head(20)

,user_id,age,gender
0,1,4,1
1,2,10,1
2,3,7,2
3,4,5,1
4,5,4,1
5,6,6,1
6,7,6,2
7,8,5,1
8,9,5,1
9,10,9,2


In [ ]:
train_feature=pd.read_pickle('./lgb_feature/train.pkl')
test_feature=pd.read_pickle('./lgb_feature/test.pkl')
feature = pd.concat([train_feature, test_feature], ignore_index=True)

In [ ]:
feature = feature.fillna(-1)
from sklearn.preprocessing import StandardScaler
ss=StandardScaler()
ss.fit(feature)
hin_feature = ss.transform(feature)
num_feature_input = hin_feature.shape[1]
gc.collect()

In [6]:
train_input_1 = x1[:len(labels)]
train_input_2 = x2[:len(labels)]
train_input_3 = x3[:len(labels)]
train_input_4 = x4[:len(labels)]
train_input_5 = x5[:len(labels)]
train_input_1 = pd.DataFrame(train_input_1)
train_input_2 = pd.DataFrame(train_input_2)
train_input_3 = pd.DataFrame(train_input_3)
train_input_4 = pd.DataFrame(train_input_4)
train_input_5 = pd.DataFrame(train_input_5)
label_train1 = pd.DataFrame(np.array(labels['age'] - 1))
label_train2 = pd.DataFrame(np.array(labels['gender'] - 1))

In [18]:
test_input_1 = x1[len(labels):]
test_input_2 = x2[len(labels):]
test_input_3 = x3[len(labels):]
test_input_4 = x4[len(labels):]
test_input_5 = x5[len(labels):]
test_input_1 = pd.DataFrame(test_input_1)
test_input_2 = pd.DataFrame(test_input_2)
test_input_3 = pd.DataFrame(test_input_3)
test_input_4 = pd.DataFrame(test_input_4)
test_input_5 = pd.DataFrame(test_input_5)

In [22]:
nn_train = pd.concat([train_input_1,train_input_2,train_input_3,train_input_4,train_input_5,label_train1,label_train2],axis=1)

In [9]:
nn_train.shape

(3000000, 502)

In [24]:
nn_train = np.array(nn_train)

In [25]:
np.save('./w2v_feature/train.npy',nn_train)

In [26]:
nn_test = pd.concat([test_input_1,test_input_2,test_input_3,test_input_4,test_input_5],axis=1)

In [13]:
np.save('./w2v_feature/test.npy',nn_test)

In [14]:
nn_test.shape

(1000000, 500)

In [ ]:
#生成h5py格式,处理的是w2v的序列特征

In [3]:
target_dir = "./target_out/"

In [4]:
nn_test = np.load(target_dir+'test.npy')

In [7]:
F1 = [i for i in range(100)]
F2 = [i + 100 for i in range(100)]
F3 = [i + 100 * 2 for i in range(100)]
F4 = [i + 100 * 3 for i in range(100)]
F5 = [i + 100 * 4 for i in range(100)]

In [8]:
nn_test1 = nn_test[:,F1]
nn_test2 = nn_test[:,F2]
nn_test3 = nn_test[:,F3]
nn_test4 = nn_test[:,F4]
nn_test5 = nn_test[:,F5]
f = h5py.File('./tfidf/test.h5', 'w')
f.create_dataset('cre', data=nn_test1)
f.create_dataset('ad', data=nn_test2)
f.create_dataset('adv', data=nn_test3)
f.create_dataset('pro', data=nn_test4)
f.create_dataset('ind', data=nn_test5)
f.close()

In [3]:
batch = 512 
f = h5py.File('./tfidf/test.h5', "r")

In [4]:
f['cre'][0:].shape


(1000000, 100)

In [ ]:
f.close()

In [3]:
F1 = [i for i in range(100)]
F2 = [i + 100 for i in range(100)]
F3 = [i + 100 * 2 for i in range(100)]
F4 = [i + 100 * 3 for i in range(100)]
F5 = [i + 100 * 4 for i in range(100)]

In [4]:
def to_ht(nn_test,flag,num):
    nn_test1 = nn_test[:,F1]
    nn_test2 = nn_test[:,F2]
    nn_test3 = nn_test[:,F3]
    nn_test4 = nn_test[:,F4]
    nn_test5 = nn_test[:,F5]
    if flag=='train':
        f = h5py.File(f'./w2v_feature/nn_feature{num}/train{num}.h5', 'w')
        f.create_dataset('cre', data=nn_test1)
        f.create_dataset('ad', data=nn_test2)
        f.create_dataset('adv', data=nn_test3)
        f.create_dataset('pro', data=nn_test4)
        f.create_dataset('ind', data=nn_test5)
        f.create_dataset('age', data=nn_test[:,-2])
        f.create_dataset('gender', data=nn_test[:,-1])
        f.close()
    if flag=='vail':
        f = h5py.File(f'./w2v_feature/nn_feature{num}/vail{num}.h5', 'w')
        f.create_dataset('cre', data=nn_test1)
        f.create_dataset('ad', data=nn_test2)
        f.create_dataset('adv', data=nn_test3)
        f.create_dataset('pro', data=nn_test4)
        f.create_dataset('ind', data=nn_test5)
        f.create_dataset('age', data=nn_test[:,-2])
        f.create_dataset('gender', data=nn_test[:,-1])
        f.close()
    gc.collect()

In [ ]:
#处理tfidf、count、target的特征，需要更换一些路径和命名

In [ ]:
target_dir = "./target_out/"

In [5]:
nn_test = np.load(target_dir+'test.npy')

In [6]:
f = h5py.File(target_dir+'test.h5', 'w')
f.create_dataset('target', data=nn_test)#count/tfidf/others...
f.close()

In [7]:
#得修改对应内容
def to_tfidf_ht(nn_test,flag,num):  
    nn_test1 = nn_test
    if flag=='train':
        f = h5py.File(target_dir +f'nn_feature{num}/train{num}.h5', 'w')
        f.create_dataset('target', data=nn_test)
        f.close()
    if flag=='vail':
        f = h5py.File(target_dir +f'nn_feature{num}/vail{num}.h5', 'w')
        f.create_dataset('target', data=nn_test)
        f.close()
    gc.collect()

In [8]:
path = target_dir+'train.npy'
nn_train = np.load(path)

In [9]:
size = 600000
nn_train1 =nn_train[:size]
nn_train2=nn_train[size:2*size]
nn_train3=nn_train[2*size:3*size]
nn_train4=nn_train[3*size:4*size]
nn_train5=nn_train[4*size:5*size]

In [12]:
print(nn_train.shape)
train1 = np.vstack((nn_train2,nn_train3,nn_train4,nn_train5))
to_tfidf_ht(train1,'train',0)
to_tfidf_ht(nn_train1,'vail',0)
del train1
train2 = np.vstack((nn_train1,nn_train3,nn_train4,nn_train5))
to_tfidf_ht(train2,'train',1)
to_tfidf_ht(nn_train2,'vail',1)
del train2
train3 = np.vstack((nn_train1,nn_train2,nn_train4,nn_train5))
to_tfidf_ht(train3,'train',2)
to_tfidf_ht(nn_train3,'vail',2)
del train3
train4 = np.vstack((nn_train1,nn_train2,nn_train3,nn_train5))
to_tfidf_ht(train4,'train',3)
to_tfidf_ht(nn_train4,'vail',3)
del train4
train5 = np.vstack((nn_train1,nn_train2,nn_train3,nn_train4))
to_tfidf_ht(train5,'train',4)
to_tfidf_ht(nn_train5,'vail',4)
del train5

(3000000, 120)


In [ ]:
#处理deepwalk得到的user_id 特征，前面的都是概率特征，这个为数值特征，得做一个归一化处理
#这里是把五份user_id已全部拼接好得到的all_user_data.npy

In [5]:
all_data = np.load('./data/all_user_data.npy')

In [14]:
scaler = MinMaxScaler()
scaler.fit(all_data)
df= scaler.transform(all_data)

In [15]:
nn_test = df[3000000:]
nn_train = df[:3000000]

In [16]:
target_dir = "./target_user/"

In [17]:
f = h5py.File(target_dir+'test.h5', 'w')
f.create_dataset('target_user', data=nn_test)#count/tfidf/others...
f.close()

In [21]:
#得修改对应内容
def to_tfidf_ht(nn_test,flag,num):  
    nn_test1 = nn_test
    if flag=='train':
        f = h5py.File(target_dir +f'nn_feature{num}/train{num}.h5', 'w')
        f.create_dataset('target_user', data=nn_test)
        f.close()
    if flag=='vail':
        f = h5py.File(target_dir +f'nn_feature{num}/vail{num}.h5', 'w')
        f.create_dataset('target_user', data=nn_test)
        f.close()
    gc.collect()

In [22]:
size = 600000
nn_train1 =nn_train[:size]
nn_train2=nn_train[size:2*size]
nn_train3=nn_train[2*size:3*size]
nn_train4=nn_train[3*size:4*size]
nn_train5=nn_train[4*size:5*size]

In [23]:
print(nn_train.shape)
train1 = np.vstack((nn_train2,nn_train3,nn_train4,nn_train5))
to_tfidf_ht(train1,'train',0)
to_tfidf_ht(nn_train1,'vail',0)
del train1
train2 = np.vstack((nn_train1,nn_train3,nn_train4,nn_train5))
to_tfidf_ht(train2,'train',1)
to_tfidf_ht(nn_train2,'vail',1)
del train2
train3 = np.vstack((nn_train1,nn_train2,nn_train4,nn_train5))
to_tfidf_ht(train3,'train',2)
to_tfidf_ht(nn_train3,'vail',2)
del train3
train4 = np.vstack((nn_train1,nn_train2,nn_train3,nn_train5))
to_tfidf_ht(train4,'train',3)
to_tfidf_ht(nn_train4,'vail',3)
del train4
train5 = np.vstack((nn_train1,nn_train2,nn_train3,nn_train4))
to_tfidf_ht(train5,'train',4)
to_tfidf_ht(nn_train5,'vail',4)
del train5

(3000000, 704)


In [ ]:
#压缩文件,传输到其他服务器

In [24]:
!zip -r target_user.zip target_user

  adding: target_user/ (stored 0%)
  adding: target_user/test.h5 (deflated 12%)
  adding: target_user/nn_feature4/ (stored 0%)
  adding: target_user/nn_feature4/train4.h5 (deflated 12%)
  adding: target_user/nn_feature4/vail4.h5 (deflated 12%)
  adding: target_user/nn_feature1/ (stored 0%)
  adding: target_user/nn_feature1/vail1.h5 (deflated 12%)
  adding: target_user/nn_feature1/train1.h5 (deflated 12%)
  adding: target_user/.ipynb_checkpoints/ (stored 0%)
  adding: target_user/nn_feature3/ (stored 0%)
  adding: target_user/nn_feature3/train3.h5 (deflated 12%)
  adding: target_user/nn_feature3/vail3.h5 (deflated 12%)
  adding: target_user/nn_feature0/ (stored 0%)
  adding: target_user/nn_feature0/train0.h5 (deflated 12%)
  adding: target_user/nn_feature0/vail0.h5 (deflated 12%)
  adding: target_user/nn_feature2/ (stored 0%)
  adding: target_user/nn_feature2/train2.h5 (deflated 12%)
  adding: target_user/nn_feature2/vail2.h5 (deflated 12%)


In [1]:
import os
os.chdir('tfidf')#w2v_feature

In [2]:
!zip -r nn_feature0.zip ./nn_feature0

  adding: nn_feature0/ (stored 0%)
  adding: nn_feature0/train0.h5 (deflated 5%)
  adding: nn_feature0/vail0.h5 (deflated 5%)


In [3]:
!zip -r nn_feature1.zip ./nn_feature1

  adding: nn_feature1/ (stored 0%)
  adding: nn_feature1/vail1.h5 (deflated 5%)
  adding: nn_feature1/train1.h5 (deflated 5%)


In [7]:
!zip -r nn_feature2.zip ./nn_feature2

  adding: nn_feature2/ (stored 0%)
  adding: nn_feature2/vail2.h5 (deflated 5%)
  adding: nn_feature2/train2.h5 (deflated 5%)


In [8]:
!zip -r nn_feature3.zip ./nn_feature3

  adding: nn_feature3/ (stored 0%)
  adding: nn_feature3/train3.h5 (deflated 5%)
  adding: nn_feature3/vail3.h5 (deflated 5%)


In [9]:
!zip -r nn_feature4.zip ./nn_feature4

  adding: nn_feature4/ (stored 0%)
  adding: nn_feature4/train4.h5 (deflated 5%)
  adding: nn_feature4/vail4.h5 (deflated 5%)
  adding: nn_feature4/.ipynb_checkpoints/ (stored 0%)


In [ ]:
#重新开启,观察数据是否生成正常

In [6]:
ps=0
filepath_tr = f"./w2v_feature/nn_feature{ps}/vail{ps}.h5"
inputPath=filepath_tr
all_num = 2400000
batch_size=1
mode="train"

In [8]:
ps = 0
f = h5py.File(f"./w2v_feature/nn_feature{ps}/train{ps}.h5", "r")
f_va = h5py.File(f"./w2v_feature/nn_feature{ps}/vail{ps}.h5", "r")

In [5]:
f.close()
#f_va.close()

In [7]:
f = h5py.File(inputPath, "r")
all_sum = all_num/batch_size
i = 0
beta = 0.2
while True:
        labels = np.zeros((batch_size, 10))
        input1 = f['cre'][i:i+batch_size]
        input2 = f['ad'][i:i+batch_size]
        input3 = f['adv'][i:i+batch_size]
        input4 = f['pro'][i:i+batch_size]
        input5 = f['ind'][i:i+batch_size]

        if mode=='train':
            for i in range(batch_size):
                num = np.random.rand()
                if num > beta:
                    lens = len(np.nonzero(input1[i])[0])
                    print(lens)
                    state = np.random.get_state()
                    np.random.shuffle(input1[i][100-lens:100-1])
                    np.random.set_state(state)
                    np.random.shuffle(input2[i][100-lens:100-1])
                    np.random.set_state(state)
                    np.random.shuffle(input3[i][100-lens:100-1])
                    np.random.set_state(state)
                    np.random.shuffle(input4[i][100-lens:100-1])
                    np.random.set_state(state)
                    np.random.shuffle(input5[i][100-lens:100-1])
            
            indexs = f['age'][i:i+batch_size]
            labels = to_categorical(indexs, num_classes=10) #one-hot
                    
            #labels = label_smoothing(labels) #smoothing
        if mode=='vail':
            indexs = f['age'][i:i+batch_size]
            labels = to_categorical(indexs, num_classes=10) #one-hot
            
        i += batch_size
        if(i==all_num):i=0  #切换下一个epoch
        ss = ([input1, input2,input3], labels)
        print(ss)
        break

13
([array([[     0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,   1123,
          3141,  17355,  82668,  54328,   2502,   1904, 369523,   1904,
         18116,     58,  30792, 150655]]), array([[     0,      0,      0,      0,      0,      0,      0,      0,
             0, 

In [8]:
ss[1].shape

(1, 10)

In [89]:
f = h5py.File(inputPath, "r")
all_sum = all_num/batch_size
i = 0
beta = 0.2
while True:
    labels = np.zeros((batch_size, 10))
    input1 = f['cre'][i:i+batch_size]
    input2 = f['ad'][i:i+batch_size]
    input3 = f['adv'][i:i+batch_size]
    input4 = f['pro'][i:i+batch_size]
    input5 = f['ind'][i:i+batch_size]
    
    if mode=='train':
        for i in range(batch_size):
            num = np.random.rand()
            if num > beta:
                state = np.random.get_state()
                np.random.shuffle(input1[i])
                np.random.set_state(state)
                np.random.shuffle(input2[i])
                np.random.set_state(state)
                np.random.shuffle(input3[i])
                np.random.set_state(state)
                np.random.shuffle(input4[i])
                np.random.set_state(state)
                np.random.shuffle(input5[i])

        indexs = f['age'][i:i+batch_size]
        labels = to_categorical(indexs, num_classes=10) #one-hot

        #labels = label_smoothing(labels) #smoothing
    if mode=='vail':
        indexs = f['age'][i:i+batch_size]
        labels = to_categorical(indexs, num_classes=10) #one-hot
        
  
        
        
    i += batch_size
    if(i==all_num):i=0  #切换下一个epoch
    print([input1,input2,input3],labels)
    print(labels)
    break

[array([[  10670,  721691,   19467,  152190,     151,      67,  102271,
              0,    2225,       0,       0,  500792,       0,   80937,
              0,       0,       0,       0,   12645,   11964,       0,
            271,       0,       0,       7,       1,       4,       0,
              0,   34460,       0,  116381,     803,     365,       0,
              0,  337598,       0,       0,   33635,    3102,       1,
              0,   12645,       0,       0,       0,    4510,   10955,
              0,    9400,  149204,  146829,       0,       0,       0,
              0,   22731,       0,       9,    4920,       0,       0,
            117,       7,   86290,   12700,     111,    2572,       0,
              0,   12645,       0,     791,       0,       0,     511,
             48,  180698,       0,  167753,       0,       0,  396644,
              0,      36,       0,       0,       4,  124711, 2514430,
             36,       0,     241,  290607,       0,       1,     187,
     

In [10]:
f.close()

In [1]:
!nvidia-smi

/bin/bash: nvidia-smi: command not found


In [ ]:
[[0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 1 0 0]
 [0 0 1 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0]]